In [1]:
!pip install rasterio
!pip install numpy
!pip install tqdm
!pip install shapely

In [2]:
conda install -c conda-forge gdal

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [36]:
!gsutil cp -r gs://hysim/S2B_T06WVT/input /home/jupyter/

Copying gs://hysim/S2B_T06WVT/input/DEMlayer.tif...
Copying gs://hysim/S2B_T06WVT/input/S2B_T06WVT.tif...                           
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://hysim/S2B_T06WVT/input/T06WVT_DEM_UTM6.tif...                      
/ [3 files][  4.6 GiB/  4.6 GiB]   83.3 MiB/s                                   
Operation completed over 3 objects/4.6 GiB.                                      


In [4]:
!gsutil cp gs://hysim/S2A_T06WVS/input/S2AL2_T06WVS.tif /home/jupyter/input/ 

Copying gs://hysim/S2A_T06WVS/input/S2AL2_T06WVS.tif...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

| [1 files][  4.5 GiB/  4.5 GiB]      0.0 B/s                                   
Operation completed over 1 objects/4.5 GiB.                                      


In [5]:
from osgeo import gdal
import numpy as np
import tempfile
import rasterio as rio
from pathlib import Path
import rasterio.mask as rio_mask
from shapely.geometry import box
from rasterio.vrt import WarpedVRT
from google.cloud import storage

In [6]:
from google.cloud import storage

storage_client = storage.Client()
buckets = storage_client.list_buckets()

for bucket in buckets:
    print(bucket.name)
    
!whoami
!pwd


alaskaepscor_simulation
hysim
jupyter
/home/jupyter


In [7]:
dem_path = Path("/home/jupyter/input/DEMlayer_WVS.tif")
src_path = Path("/home/jupyter/input/S2AL2_T06WVS.tif")
#dst_path = "gs://testbucket_111221/output/DEMlayer_B.tif"

In [8]:
dst_path = "/home/jupyter/input/DEMlayer.tif"

In [9]:
#without slope and aspect=============
"""
with rio.open(src_path, 'r') as img, rio.open(dem_path, 'r') as dem:
    with WarpedVRT(dem, crs=img.crs) as dem_vrt:
        meta = dem_vrt.meta.copy()
        bbox = box(*img.bounds)
        dst_img, dst_transform = rio_mask.mask(
          dataset=dem_vrt,
          shapes=(bbox,),
          invert=False,
          all_touched=False,
          crop=True,
          filled=True
        )
        meta['driver'] = 'GTiff'
        meta['count'], meta['height'], meta['width'] = dst_img.shape
        meta['transform'] = dst_transform
        with rio.open(dst_path, 'w', **meta) as dst:
            dst.write(dst_img)
 """

"\nwith rio.open(src_path, 'r') as img, rio.open(dem_path, 'r') as dem:\n    with WarpedVRT(dem, crs=img.crs) as dem_vrt:\n        meta = dem_vrt.meta.copy()\n        bbox = box(*img.bounds)\n        dst_img, dst_transform = rio_mask.mask(\n          dataset=dem_vrt,\n          shapes=(bbox,),\n          invert=False,\n          all_touched=False,\n          crop=True,\n          filled=True\n        )\n        meta['driver'] = 'GTiff'\n        meta['count'], meta['height'], meta['width'] = dst_img.shape\n        meta['transform'] = dst_transform\n        with rio.open(dst_path, 'w', **meta) as dst:\n            dst.write(dst_img)\n "

In [10]:
#NEw======
with rio.open(src_path, 'r') as img, rio.open(dem_path, 'r') as dem:
    with WarpedVRT(dem, crs=img.crs) as dem_vrt:
        meta = dem_vrt.meta.copy()
        bbox = box(*img.bounds)
        dst_img, dst_transform = rio_mask.mask(
          dataset=dem_vrt,
          shapes=(bbox,),
          invert=False,
          all_touched=False,
          crop=True,
          filled=True
        )
        meta['driver'] = 'GTiff'
        meta['count'], meta['height'], meta['width'] = dst_img.shape
        meta['transform'] = dst_transform
        _, dem_file = tempfile.mkstemp(text=False)
        _, slope_file = tempfile.mkstemp(text=False)
        _, aspect_file = tempfile.mkstemp(text=False)

        with rio.open(dem_file, 'w', **meta) as dst:
          dst.write(dst_img)
        ds = gdal.DEMProcessing(slope_file, dem_file, 'slope')
        ds = None
        ds = gdal.DEMProcessing(aspect_file, dem_file, 'aspect')
        ds = None
        with rio.open(dem_file, 'r') as dem, rio.open(slope_file, 'r') as slope, rio.open(aspect_file, 'r') as aspect:
          arr = np.concatenate([dem.read(), slope.read(), aspect.read()], axis=0)
        Path(dem_file).unlink()
        Path(slope_file).unlink()
        Path(aspect_file).unlink()
        meta['count'] = 3
        with rio.open(dst_path, 'w', **meta) as dst:
          dst.write(arr) 

In [21]:
#!gsutil cp -r /home/jupyter/output/ gs://testbucket_111221

Copying file:///home/jupyter/output/DEMlayer_B.tif [Content-Type=image/tiff]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1 files][271.4 MiB/271.4 MiB]                                                
Operation completed over 1 objects/271.4 MiB.                                    


In [25]:
#!rm -rf /home/jupyter/output/